In [1]:
import numpy as np
from matplotlib import pyplot as plt
import uproot, uproot_methods
import uproot_methods.classes.TLorentzVector as TLorentzVector

In [6]:
import ROOT

ModuleNotFoundError: No module named 'ROOT'

In [5]:
#include <TH1D.h>
include <TChain.h>
#include <TFile.h>

SyntaxError: invalid syntax (<ipython-input-5-768ed4d89697>, line 2)

In [2]:
class quickReco:
    
    def __init__(self, name='', inputfile='', isTest=False):
        
        self.name = name
        self.delphesFile= uproot.open(inputfile)['Delphes']
        self.isTestRun = isTest
        
        # thresholds
        self.minLepPt  = 15
        self.maxLepEta = 2.5
        self.minJetPt  = 20
        self.maxJetEta = 2.5
    
        # variables unique for each event
        self.selectedMuons = []
        self.selectedElectrons = []
        self.selectedJets = []
        
        # variables to keep track
        self.nJets      = []
        self.nElectrons = []
        self.nMuons     = []
        self.HT         = []
        self.missingET  = []
        self.dileptonMass = []
        self.dileptonPT   = []
        self.dilepton_Jets = []
        self.jetCutflow = {'all':0, 'pt>{}'.format(self.minJetPt):0, '|eta|<{}'.format(self.maxJetEta):0}
        self.electronCutflow = {'all':0, 'pt>{}'.format(self.minLepPt):0, '|eta|<{}'.format(self.maxLepEta):0}
        self.muonCutflow = {'all':0, 'pt>{}'.format(self.minLepPt):0, '|eta|<{}'.format(self.maxLepEta):0}
        self.eventCutflow = {'all':0, 'El: >=2 lep':0, 'El: ==2 OS lep':0, 'El: Z window':0, 'Mu: >=2 lep':0, 'Mu: ==2 OS lep':0, 'Mu: Z window':0}
        
        # Branch Definitions
        self.l_jetPt     = []
        self.l_jetEta    = []
        self.l_jetPhi    = []
        self.l_jetMass   = []
        self.l_elPt      = []
        self.l_elEta     = []
        self.l_elPhi     = []
        self.l_elCharge  = []
        self.l_muPt      = []
        self.l_muEta     = []
        self.l_muPhi     = []
        self.l_muCharge  = []
        self.l_missingET_met  = []
        self.l_missingET_phi  = []
        self.l_scalarHT       = []
        
        
    def process(self):
        """main function for processing files"""
        
        # *** 0. Load Branches
        self.loadBranches()
        
        # *** 1. Loop over events
        for iEvent in range(0, len(self.delphesFile)):
           
            #keep track of events
            if iEvent%500 == 0:
                print("Processed {} events...".format(iEvent))
            
            #skip out if testRun
            if self.isTestRun and iEvent > 10:
                continue
            
            # ** A. Re-initialize collections
            self.selectedElectrons = []
            self.selectedMuons = []
            self.selectedJets = []
            
            # ** B. Object selection
            self.leptonSelection('Electron', iEvent)
            self.leptonSelection('Muon', iEvent)
            self.jetSelection(iEvent)

            # ** C. Event selection
            self.eventSelection(iEvent)
            

       # *** 2. Print some summaries
    
    
    def eventSelection(self, _iEvent):
        """function to select events"""
        
        #print("Evt: {}, N_electrons= {}, N_muons= {}".format(_iEvent, len(self.selectedElectrons), len(self.selectedMuons)))
        self.eventCutflow['all'] += 1
        
        if len(self.selectedElectrons) > 2 and len(self.selectedMuons)>2:
            print(">2 muons AND >2 electrons in Event {}. Ambiguous, so skipping...".format(_iEvent))
            return
        
        # ** 1. electron channel
        self.dileptonReco(self.selectedElectrons, 'electron', _iEvent)

        # ** 2. muon channel
        self.dileptonReco(self.selectedMuons, 'muon', _iEvent)

    def dileptonReco(self, _leptons, flavor, _iEvent):
        """helper function for dilepton reconstruction"""
        
        cutflowTag = 'El' if flavor=='electron' else 'Mu'
        
        if len( _leptons) >= 2:
            self.eventCutflow['{}: >=2 lep'.format(cutflowTag)] += 1
            
            if len(_leptons) == 2:
                lep0 = _leptons[0]
                lep1 = _leptons[1]
        
                if lep0[1]*lep1[1] == -1:
                    self.eventCutflow['{}: ==2 OS lep'.format(cutflowTag)] += 1
                    
                    m_ll = ( lep0[0] + lep1[0] ).mass
                    pt_ll = ( lep0[0] + lep1[0] ).pt
                    
                    self.dileptonMass.append( m_ll)
                    self.dileptonPT.append( pt_ll)
                    self.dilepton_Jets.append(self.nJets[_iEvent])
            
            elif len(_leptons) >2:
                print("Event {} has >2 {}. print (pT, charge)...".format(_iEvent, flavor))
                for lep in _leptons:
                    print("pt: {}, charge: {}".format(lep[0].pt, lep[1]))
                return
            
        else:
            return
            
    def leptonSelection(self, flavor, _iEvent):
        """function for performing lepton selection"""
        
        _selectedLeptons = []
        
        _pt     = self.l_elPt if flavor=='Electron' else self.l_muPt 
        _eta    = self.l_elEta if flavor=='Electron' else self.l_muEta 
        _phi    = self.l_elPhi if flavor=='Electron' else self.l_muPhi
        _charge = self.l_elCharge if flavor=='Electron' else self.l_muCharge 
        _mass   = 0.511 * 1e-3 if flavor == 'Electron' else 105.6*1e-3 # need conversion 1e-3 for MeV to GeV
        
        for iLep in range(0, len(_pt[_iEvent])): 
            if flavor=='Electron':
                self.electronCutflow['all'] += 1
            elif flavor=='Muon':
                self.muonCutflow['all'] += 1
             
            # ** A. Check pt above min threshold
            if _pt[_iEvent][iLep] < self.minLepPt:
                continue
                
            if flavor=='Electron':
                self.electronCutflow['pt>{}'.format(self.minLepPt)] += 1
            elif flavor=='Muon':
                self.muonCutflow['pt>{}'.format(self.minLepPt)] += 1
                
            # ** B. Check eta inside max abs eta range
            if np.abs(_eta[_iEvent][iLep]) > self.maxLepEta:
                continue

            if flavor=='Electron':
                self.electronCutflow['|eta|<{}'.format(self.maxLepEta)] += 1
            elif flavor=='Muon':
                self.muonCutflow['|eta|<{}'.format(self.maxLepEta)] += 1
            
            # ** C. Make LorentzVector and store
            _tlv = TLorentzVector.PtEtaPhiMassLorentzVector( _pt[_iEvent][iLep], _eta[_iEvent][iLep], _phi[_iEvent][iLep], _mass)
            _selectedLeptons.append( (_tlv, _charge[_iEvent][iLep]) )
            
            
        # some global stuff after selection        
        if flavor=='Electron':
            self.selectedElectrons = _selectedLeptons
            self.nElectrons.append( len(_selectedLeptons))
        elif flavor=='Muon':
            self.selectedMuons = _selectedLeptons
            self.nMuons.append( len(_selectedLeptons))
                
                
    def jetSelection(self, _iEvent):
        """function for performing jet selection"""
        
        _selectedJets = []
        
        _pt     = self.l_jetPt  
        _eta    = self.l_jetEta 
        _phi    = self.l_jetPhi
        _mass   = self.l_jetMass

        for iJet in range(0, len(_pt[_iEvent])): 
            self.jetCutflow['all'] += 1
             
            # ** A. Check pt above min threshold
            if _pt[_iEvent][iJet] < self.minJetPt:
                continue
                
            self.jetCutflow['pt>{}'.format(self.minJetPt)] += 1
                
            # ** B. Check eta inside max abs eta range
            if np.abs(_eta[_iEvent][iJet]) > self.maxJetEta:
                continue

            self.jetCutflow['|eta|<{}'.format(self.maxJetEta)] += 1
            
            # ** C. Make LorentzVector and store
            _tlv = TLorentzVector.PtEtaPhiMassLorentzVector( _pt[_iEvent][iJet], _eta[_iEvent][iJet], _phi[_iEvent][iJet], _mass[_iEvent][iJet])
            _selectedJets.append( _tlv )
            
        # some global stuff after selection        
        self.selectedJets = _selectedJets
        self.nJets.append( len(_selectedJets))
       

        
    def loadBranches(self):
        """function for loading branches from delphes file"""
            
        # branches
        self.l_jetPt   = uproot.tree.TBranchMethods.array(self.delphesFile['Jet']['Jet.PT']).tolist()
        self.l_jetEta  = uproot.tree.TBranchMethods.array(self.delphesFile['Jet']['Jet.Eta']).tolist()
        self.l_jetPhi  = uproot.tree.TBranchMethods.array(self.delphesFile['Jet']['Jet.Phi']).tolist()
        self.l_jetMass = uproot.tree.TBranchMethods.array(self.delphesFile['Jet']['Jet.Mass']).tolist()
        
        self.l_elPt     = uproot.tree.TBranchMethods.array(self.delphesFile['Electron']['Electron.PT']).tolist()
        self.l_elEta    = uproot.tree.TBranchMethods.array(self.delphesFile['Electron']['Electron.Eta']).tolist()
        self.l_elPhi    = uproot.tree.TBranchMethods.array(self.delphesFile['Electron']['Electron.Phi']).tolist()
        self.l_elCharge = uproot.tree.TBranchMethods.array(self.delphesFile['Electron']['Electron.Charge']).tolist()
       
        self.l_muPt     = uproot.tree.TBranchMethods.array(self.delphesFile['Muon']['Muon.PT']).tolist()
        self.l_muEta    = uproot.tree.TBranchMethods.array(self.delphesFile['Muon']['Muon.Eta']).tolist()
        self.l_muPhi    = uproot.tree.TBranchMethods.array(self.delphesFile['Muon']['Muon.Phi']).tolist()
        self.l_muCharge = uproot.tree.TBranchMethods.array(self.delphesFile['Muon']['Muon.Charge']).tolist()
        
        self.l_missingET_met  = uproot.tree.TBranchMethods.array(self.delphesFile['MissingET']['MissingET.MET']).tolist()
        self.l_missingET_phi  = uproot.tree.TBranchMethods.array(self.delphesFile['MissingET']['MissingET.Phi']).tolist()
        self.l_scalarHT       = uproot.tree.TBranchMethods.array(self.delphesFile['ScalarHT']['ScalarHT.HT']).tolist()
        
        print("Finished loading branches...")

In [4]:
*tree = new TChain ("T")

SyntaxError: invalid syntax (<ipython-input-4-63be75cbeb9e>, line 1)